In [629]:
import sys
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import gensim

In [630]:
sys.path.append('../')

In [631]:
from src import settings

In [632]:
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [633]:
from src.models import make_texts_corpus
from src.utils import editorJs_data_to_text
import json
from gensim.matutils import cossim
from gensim import similarities

In [634]:
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
lda_model = gensim.models.ldamodel.LdaModel.load('../models/LDA.model')

INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2022-12-29T04:02:50.212335', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 177 documents, 4087 features, 36185 non-zero entries
INFO : loading LdaModel object from ../models/LDA.model
INFO : loading expElogbeta from ../models/LDA.model.expElogbeta.npy with mmap=None
INFO : setting ignored attribute state to None
INFO : setting ignored attribute dispatcher to None
INFO : setting ignored attribute id2word to None
INFO : LdaModel lifecycle event {'fname': '../models/LDA.model', 'datetime': '2022-12-29T04:02:50.268332', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.

In [635]:
index = similarities.MatrixSimilarity(lda_model[corpus])

INFO : creating matrix with 177 documents and 57 features


In [636]:
for post in col.find():
    print(post['title'], post['idrs'] if 'idrs' in post else False)

ĐỒ ÁN THIẾT KẾ WEBSITE BLOG SITE 0
setState trong React 1
Regular Expression trong Javascript 2
Các bài tập java cơ bản 3
Những câu hỏi lập trình khi pv (2): bài toán ATM 4
Ví dụ sắp xếp nổi bọt với Java 5
Chương trình lọc thư rác bằng Java, code ví dụ 6
Đọc ghi file Excel trong Java sử dụng Apache POI 7
THREAD AND THREAD POOLS IN JAVA 8
Sử dụng CountDownLatch trong Java 9
Multithreading: Java Synchronized Blocks 10
Understand synchronized and improve lock performance in Java 11
Front-End Web Developers 12
Material Design for Web Design 13
Những kỹ năng cần phải có của một Web Designer 14
Rest API - cách Frontend và Backend nói chuyện với nhau qua ví dụ cụ thể 15
Điều cần chú ý để xây dựng RESTful API 16
Dựng một API RESTful đơn giản sử dụng Express và MongoDB 17
xây dựng API với Napa 18
Trí tuệ nhân tạo với Swift 19
Lợi ích và nguy cơ của trí tuệ nhân tạo (AI) 20
Liệu chúng ta có thể xây dựng trí tuệ nhân tạo mà không mất kiểm soát đối với nó? 21
Agile Marketing khái niệm và tại sao n

In [662]:
# main_post = col.find_one({"_id": ObjectId("6388c04363cdee37de74f09c")})
main_post = col.find_one({"idrs": 165})
main_post['title']

'Văn hóa Tây Bắc - những mạch nguồn chảy mãi'

In [663]:
content = editorJs_data_to_text(json.loads(main_post["content"]))
text_corpus = make_texts_corpus([content])
bow = id2word.doc2bow(next(text_corpus))

In [664]:
vector_doc = lda_model[bow]

In [665]:
sims = index[vector_doc]

In [666]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#for doc_position, doc_score in sims:
#    print(doc_score, doc_position)

In [667]:
print(main_post['title'])
i=0
for idrs, distance in sims:
    article = col.find_one({"idrs": idrs})
    print(i, idrs, distance, article['title'])
    i+=1
    if(i==10): break
    

Văn hóa Tây Bắc - những mạch nguồn chảy mãi
0 165 1.0 Văn hóa Tây Bắc - những mạch nguồn chảy mãi
1 166 0.93412447 Sắc màu văn hóa các dân tộc vùng Tây Bắc
2 110 0.8781304 Chuẩn bị đi du lịch? Cẩm nang từ A-Z giúp bạn có kỳ nghỉ TRỌN VẸN
3 163 0.8479252 Áo dài - nét đẹp đặc sắc của văn hoá Việt Nam
4 161 0.80711895 Chiếc áo dài – Nét đẹp văn hóa truyền thống của người phụ nữ Việt Nam
5 162 0.79013026 Lịch sử phát triển áo dài Việt Nam qua các thời kỳ
6 107 0.7535104 Những phong tục trong ngày Tết cổ truyền
7 29 0.7342495 Chia sẻ kinh nghiệm du lịch bụi đơn giản nhưng rất cần thiết cho du khách
8 164 0.7148565 Ý nghĩa tà áo dài truyền thống Việt Nam – giá trị qua các thời kỳ lịch sử
9 109 0.7083704 Giữ gìn, phát huy nét đẹp văn hóa Tết cổ truyền của Dân tộc Việt Nam


In [668]:
arr = [tup[0] for tup in sims[:10]]
if 'idrs' in main_post: 
    if main_post['idrs'] in arr: arr.remove(main_post['idrs'])
print(main_post['idrs'])
list(arr)

165


[166, 110, 163, 161, 162, 107, 29, 164, 109]